# Proyecto final: análisis de datos aplicado a una nueva bebida energizante

## Contenido

1. [Planteamiento del problema](#sec_1)
2. [Exploración del set de datos](#sec_2)
3. [Limpieza del set de datos](#sec_3)
4. [Análisis de los datos](#sec_4)
5. [Reporte final y recomendaciones](#sec_5)
6. [Sugerencias finales](#sec_6)

<a id='sec_1'></a>
## 1. Planteamiento del problema

*MegaBoost* es una marca de bebidas energizantes de Inglaterra y desea ingresar al mercado Italiano. Hace algunos meses lanzaron su bebida en 10 ciudades de Italia.

El equipo de marketing está a cargo de incrementar el posicionamiento de la marca y la participación en el mercado, además de apoyar el desarrollo del producto. Para esto aplicaron una encuesta a 10.000 personas en esas 10 ciudades.

Suponga que usted es parte del equipo de marketing encargado del análisis de datos y tiene la tarea de tomar los resultados de esta encuesta y extraer información relevante (**insights**) para la toma de decisiones con respecto al producto.

### 1.1. Tarea a realizar

Su labor es extraer los siguientes *insights* a partir de los datos:

- **Demográficos:**
    - ¿Quiénes prefieren esta bebida energética?
    - ¿Qué rangos de edad tienden a preferir la bebida?
    
 
- **Análisis de la competencia:**
    - ¿Quiénes son los actuales líderes del mercado?

- **Canales de mercadeo y conocimiento de la marca:**
    - ¿Cuál es el canal de mercadeo más efectivo?
    
- **Penetración de la marca:**
    - ¿Qué piensa la gente de nuestra marca?
    - ¿En qué ciudades debemos reforzar este posicionamiento?
    - ¿Por qué los consumidores prefieren otras marcas y no las nuestras?

Al final, y con base en el análisis anterior:

- Genere un reporte muy breve con los principales hallazgos
- Dé al menos 3 recomendaciones al equipo de marketing para mejorar las ventas del producto.

### 1.2. Recursos disponibles

- [Formato de la encuesta aplicada a las 10.000 personas](encuesta_bebida_energizante.pdf)
- [Set de datos](dataset_bebida_energizante.csv)
- [Metadatos con la descripción de la información contenida en el set de datos](metadatos_dataset_bebida_energizante.txt)

<a id='sec_2'></a>
## 2. Exploración del set de datos

El objetivo de esta fase es familiarizarnos con la información contenida en el set de datos.

Aunque cada proyecto es diferente sugiero llevar a cabo al menos estos pasos de exploración:

1. Leer el set de datos
2. Entender tamaño, columnas y filas del set de datos
3. Entender qué tipos de datos tenemos en cada columna
4. Entender el rango de valores de las variables numéricas
5. Entender los niveles de las diferentes variables categóricas
6. Verificar si hay datos faltantes

Es importante que al final de cada fase, y en caso de que sea pertinente, agreguemos una breve síntesis de lo observado. Esto nos permitirá mantener una bitácora de las observaciones que vayamos encontrando y será útil en fases posteriores del análisis.

Llevemos a cabo cada una de estas fases:

### 2.1. Leer el set de datos
y
### 2.2. Entender su tamaño y el contenido de sus filas y columnas

In [ ]:
import pandas as pd
df = pd.read_csv('dataset_bebida_energizante.csv')
df

> **Observación**
> Tenemos 10.000 filas, cada una correspondiente a una de las personas encuestadas y 28 columnas con la información de la encuesta y con los identificadores mencionados en los metadatos

### 2.3. Entender qué tipos de datos tenemos en cada columna

In [ ]:
df.info()

Prácticamente todas las columnas contienen datos de tipo categórico exceptuando aquellas que comienzan con "ID_". Sin embargo, la columna "Experiencia_sabor" corresponde a las respuestas dadas por los encuestados a una pregunta, pero está en formato numérico.

Exploremos en detalle esta columna:

In [ ]:
df['Experiencia_sabor'].value_counts()

Vemos que en realidad no es una columna numérica. Se trata de una variable categórica ordinal con la calificación dada por los encuestados al sabor de la bebida (1, 2, 3, 4 o 5 siendo 5 el puntaje más alto).

Podemos cambiar el tipo de dato de esta columna de "int64" a "Object" para evitar este inconveniente y en adelante poderla tratar como variable categórica:

In [ ]:
df['Experiencia_sabor'] = df['Experiencia_sabor'].astype('object')

> **Observaciones**
> - La mayoría de las columnas son de tipo categórico exceptuando las columnas ID_Ciudad, ID_respuesta que son de tipo numérico entero

### 2.4. Entender el rango de valores de las variables numéricas

In [ ]:
# Esta opción selecciona todas las columnas numéricas
df.select_dtypes(include='number')

Podríamos usar `describe()` para ver en detalle la distribución de estas columnas. Sin embargo, en este caso esta opción no es relevante pues las columnas no contienen información que sea importante para las posteriores etapas de análisis.

> **Observación**
> Las columnas "ID_encuestado" y "ID_respuesta" no contienen información relevante para el análisis que nos interesa y pueden ser eliminadas en etapas posteriores

### 2.5. Entender los niveles de las diferentes variables categóricas

En este caso resulta muy útil el método `value_counts()`. Lo que haremos será iterar sobre las diferentes columnas categóricas para imprimir en pantalla los diferentes niveles:

In [ ]:
# Iterar sobre cada columna categórica y en cada iteración imprimir en pantalla
# los niveles obtenidos con "value_counts"

for column in df.select_dtypes(exclude='number').columns:
    print('-'*30)
    print(f'*** {column} ***')
    print(df[column].value_counts())
    print('-'*30)
    input('Presione enter para continuar...')
    print('')
        

> **Observaciones**
> - La columna "ID_Ciudad" no contiene información relevante y podremos eliminarla en etapas posteriores

### 2.6. Verificar si hay datos faltantes

Podemos usar de nuevo el método `info()` para tomar nota de las columnas que contienen datos faltantes:

In [ ]:
df.info()

> **Observaciones**
> - Las columnas "Edad", "Frec_consumo", "Motivo_consumo" y "Había probado" contienen 12 datos faltantes cada una
> - Por tratarse de columnas que resultan relevantes para nuestro análisis **NO** las eliminaremos

<a id='sec_3'></a>
## 3. Limpieza del set de datos

De nuevo, cada dataset y cada proyecto tendrá sus particularidades, pero sugiero al menos estas etapas de limpieza:

1. Eliminar columnas irrelevantes
2. Reorganizar las columnas relevantes
3. Manejo de datos faltantes

Implementemos cada fase teniendo en cuenta las observaciones que hemos hecho hasta el momento.

### 3.1. Eliminar columnas irrelevantes

De las observaciones hechas anteriormente concluimos que se pueden eliminar estas columnas: "ID_encuestado", "ID_respuesta" y "ID_Ciudad".

Esto lo haremos con el método `drop`:

In [ ]:
df = df.drop(columns = ['ID_encuestado','ID_respuesta','ID_Ciudad'])
df

### 3.2. Reorganizar las columnas relevantes

En este punto tenemos únicamente columnas de tipo categórico. La idea es organizarlas por orden de importancia.

Pero además de esto, si observamos la encuesta de referencia y los metadatos, podremos agrupar las columnas por categorías:

**Columnas con información demográfica**
- Edad
- Género
- Ciudad
- Cat_Ciudad (esta columna no hace parte de la encuesta pero fue añadida por el equipo de marketing)

**Columnas con hábitos de consumo**
- Frec_consumo
- Momento_consumo
- Motivo_consumo
- Situaciones_típicas_consumo

**Conocimiento de la marca**
- Conocía_marca
- Percepción_marca
- Percepción_general

**Conocimiento del producto**
- Había_probado
- Experiencia_sabor
- Motivos_no_probarla

**Competencia**
- Marcas_que_consume
- Motivos_escoger_marca
- Mejoras_sugeridas

**Salud e ingredientes**
- Ingredientes_esperados
- Preocupaciones_salud'
- Interés_en_natural_o_organico

**Mercadeo**
- Canales_marketing
- Preferencia_empaque
- Empaque_ed_limitada
- Rango_precios
- Lugar_compra

In [ ]:
# Grupos de columnas
cols_demog = ['Edad', 'Género', 'Ciudad', 'Cat_Ciudad']
cols_habit = ['Frec_consumo', 'Momento_consumo', 'Motivo_consumo', 'Situaciones_típicas_consumo']
cols_marca = ['Conocía_marca', 'Percepción_marca', 'Percepción_general']
cols_prodc = ['Había_probado', 'Experiencia_sabor', 'Motivos_no_probarla']
cols_compe = ['Marcas_que_consume', 'Motivos_escoger_marca', 'Mejoras_sugeridas']
cols_salud = ['Ingredientes_esperados', 'Preocupaciones_salud', 'Interés_en_natural_o_organico']
cols_markt = ['Canales_marketing', 'Preferencia_empaque', 'Empaque_ed_limitada', 'Rango_precios', 'Lugar_compra']

In [ ]:
# Reorganizar columnas por orden de prioridad
col_order = cols_demog + cols_marca + cols_prodc +  cols_compe + cols_markt + cols_habit   + cols_salud
df = df[col_order]
df

### 3.3. Manejo de datos faltantes

En nuestro caso todos los datos son categóricos. En este caso usualmente tenemos estas alternativas:

1. **Eliminar la fila completa**: puede ser adecuada si son pocos los datos faltantes y nuestro set contiene muchas más filas
2. **Reemplazar los datos faltantes con el valor más frecuente**: esto podría generar un sesgo aún mayor en la distribución de los niveles para la categoría de interés
3. **Desarrollar un modelo que prediga los datos faltantes**: es decir que tome información de los datos (filas y columnas) que están completos para predecir la variable en la celda faltante.
4. **Eliminar la columna**: se puede llevar a cabo si la columna de interés no contiene información relevante para el análisis

En este caso podrían ser la opción (1) o (2). Verifiquemos nuevamente la cantidad de datos faltantes con `info()`:

In [ ]:
df.info()

Las filas "problemáticas" tienen tan sólo 12 datos faltantes, muy pocos comparados con los 10.000 registros. Optaremos por la opción 1, usando `dropna()`:

In [ ]:
df = df.dropna()
df.info()

<a id='sec_4'></a>
## 4. Análisis de los datos

En este caso debemos regresar al planteamiento inicial del problema y a los *insights* que queremos encontrar.

Realizaremos el análisis enfocándonos de manera individual en cada grupo de *insights*. De nuevo, es clave que al final de cada análisis escribamos una breve observación que servirá como insumo para el reporte final.

### 4.1. Análisis demográfico

En este análisis buscamos responder las preguntas:

- ¿Quiénes prefieren esta bebida energética?
- ¿Qué rangos de edad tienden a preferir la bebida?

Para responder estas preguntas podemos usar la información de las columnas demográficas (`cols_demog`):

In [ ]:
df[cols_demog]

Podemos analizar las columnas "Edad", "Género" y "Ciudad" usando `value_counts()`:

In [ ]:
print('Distribución edades:')
print(df['Edad'].value_counts())
print('-'*20)
print('Distribución géneros:')
print(df['Género'].value_counts())
print('-'*20)
print('Distribución ciudades:')
print(df['Ciudad'].value_counts())

En ocasiones el conteo no facilita la interpretación y puede resultar más adecuado usar porcentajes. Podemos seguir usando `value_counts` pero con el argumento `normalize = True`:

In [37]:
print('Distribución edades:')
print(df['Edad'].value_counts(normalize=True))
print('-'*20)
print('Distribución géneros:')
print(df['Género'].value_counts(normalize=True))
print('-'*20)
print('Distribución ciudades:')
print(df['Ciudad'].value_counts(normalize=True))

Distribución edades:
19-30    0.551962
31-45    0.237485
15-18    0.148879
46-65    0.042651
65+      0.019023
Name: Edad, dtype: float64
--------------------
Distribución géneros:
Hombre        0.603925
Mujer         0.345414
No binario    0.050661
Name: Género, dtype: float64
--------------------
Distribución ciudades:
Turín        0.282739
Venecia      0.183020
Roma         0.150981
Florencia    0.093813
Génova       0.090709
Palermo      0.056668
Verona       0.045555
Milán        0.042952
Pisa         0.036043
Siena        0.017521
Name: Ciudad, dtype: float64


Pero además debemos asegurarnos que el análisis se realiza únicamente sobre las personas que realmente conocen prefieren nuestra bebida, es decir para las cuales la columna "Marcas_que_consume" tiene un valor de "MegaBoost".

Así que debemos usar condicionales para poder extraer la información relevante:

In [38]:
df['Marcas_que_consume'].value_counts()

Coca Cola    2536
Pepsi        2109
Monster      1851
Red Bull     1058
HyperFuel     979
MegaBoost     976
Otras         479
Name: Marcas_que_consume, dtype: int64

In [39]:
print('Distribución edades:')
print(df[df['Marcas_que_consume']=='MegaBoost']['Edad'].value_counts(normalize=True))
print('-'*20)
print('Distribución géneros:')
print(df[df['Marcas_que_consume']=='MegaBoost']['Género'].value_counts(normalize=True))
print('-'*20)
print('Distribución ciudades:')
print(df[df['Marcas_que_consume']=='MegaBoost']['Ciudad'].value_counts(normalize=True))

Distribución edades:
19-30    0.550205
31-45    0.246926
15-18    0.138320
46-65    0.037910
65+      0.026639
Name: Edad, dtype: float64
--------------------
Distribución géneros:
Hombre        0.602459
Mujer         0.358607
No binario    0.038934
Name: Género, dtype: float64
--------------------
Distribución ciudades:
Turín        0.299180
Venecia      0.183402
Roma         0.158811
Florencia    0.094262
Génova       0.094262
Palermo      0.049180
Verona       0.046107
Milán        0.040984
Pisa         0.028689
Siena        0.005123
Name: Ciudad, dtype: float64


> **Respuestas preguntas análisis demográfico**

> ¿Quiénes prefieren esta bebida energética?
> - El género que más consume son los hombres (60%) seguidos por las mujeres (36%) y por el género no binario (4%) y las ciudades top-3 para los 3 géneros son Turín, Venecia y Roma.

> ¿Qué rangos de edad tienden a preferir la bebida?
> - El grupo de edades que más consume es de 19 a 30 años (55%) seguido por 31-45 (24%)

> **Aclaración:** también podríamos hacer el análisis de múltiples variables de manera simultánea usando métodos de agrupamiento, pero estas técnicas las veremos en el próximo curso.

### 4.2. Análisis de la competencia

La pregunta que queremos resolver es: ¿Quiénes son los actuales líderes del mercado?

Esta pregunta se puede responder de manera sencilla analizando la columna `Marcas_que_consume`:

In [40]:
df['Marcas_que_consume'].value_counts(normalize=True)

Coca Cola    0.253905
Pepsi        0.211153
Monster      0.185322
Red Bull     0.105927
HyperFuel    0.098018
MegaBoost    0.097717
Otras        0.047958
Name: Marcas_que_consume, dtype: float64

> **Respuesta preguna análisis de la competencia**

> - ¿Quiénes son los actuales líderes del mercado?
> - Coca Cola (25%), Pepsi (21%), Monster (19%).
> - Nuestra marca tiene un porcentaje de participación del 10%

### 4.3. Penetración de la marca

Las preguntas a responder y las columnas que podríamos usar son:

- ¿Qué piensa la gente de nuestra marca?: columnas `Conocía_marca` + `Había_probado`+ `Experiencia_sabor`, `Conocía_marca` + `Percepción_marca`, `Conocía_marca` + `Motivos_no_probarla`
- ¿En qué ciudades debemos reforzar este posicionamiento?: ciudades con menos conocimiento de marca (`Ciudad` + `Conocía_marca`) 
- ¿Por qué los consumidores prefieren otras marcas y no las nuestras? `Motivos_escoger_marca`

In [41]:
# ¿Qué piensa la gente de nuestra marca?
# Análisis combinado "Conocía_marca" + 'Había_probado' + 'Experiencia_sabor'
df[(df['Conocía_marca']=='Sí') & (df['Había_probado']=='Sí')]['Experiencia_sabor'].value_counts(normalize=True)

3    0.306133
4    0.252720
5    0.190406
2    0.147379
1    0.103363
Name: Experiencia_sabor, dtype: float64

> **Observaciones**:
> Del total de quienes conocen la marca y han probado la bebida:
> - El 30% le da una puntuación de 3 (igual que otras bebidas de otras marcas)
> - El 25% le da una puntuación de 4 (me gustó)
> - El 20% le da una puntuación de 5 (me fascinó)

In [42]:
# ¿Qué piensa la gente de nuestra marca?
# Análisis combinado 'Conocía_marca' + 'Percepción_marca'
df[df['Conocía_marca']=='Sí']['Percepción_marca'].value_counts(normalize=True)

Neutral     0.575901
Positiva    0.242568
Negativa    0.181532
Name: Percepción_marca, dtype: float64

> **Observaciones**: del total de quienes conocen la marca:
> - El 58% tiene una percepción neutral acerca del nombre y del logo
> - El 24% tiene una percepción positiva acerca del nombre y del logo

In [43]:
# ¿Qué piensa la gente de nuestra marca?
# Análisis combinado 'Conocía_marca' + 'Motivos_no_probarla'
df[df['Conocía_marca']=='Sí']['Motivos_no_probarla'].value_counts(normalize=True)

No disponible en la ciudad                 0.237387
Preocupaciones de salud                    0.225000
No me interesan las bebidas energéticas    0.220045
No conozco la marca                        0.198649
Otro                                       0.118919
Name: Motivos_no_probarla, dtype: float64

> **Observaciones**: del total de encuestados que conocen la marca, los motivos para no probar la bebida fueron:
> 1. Porque no está disponible en la ciudad: 24%
> 2. Porque existen preocupaciones sobre los efectos que pueda tener en la salud: 23%
> 3. Porque no le interesan las bebidas energéticas: 22%

> No hay un motivo único que nos permita entender los motivos por los cuales la gente no ha probado la bebida.

Y podemos combinar las anteriores observaciones para responder a la pregunta ¿qué piensa la gente de nuestra marca?:

> En general la mayoría de los encuestados que conoce nuestra marca tiene una percepción neutral de la misma (58%). Con respecto al sabor, el 55% le da una puntuación media o baja (entre 1 y 3 en una escala de 5) y el 45% le da una puntuación alta (4 o 5).

Lo anterior nos permite concluir que nuestra marca y nuestro logo no están generando la recordación esperada en los compradores y que se deberían mejorar aspectos relacionados con el sabor.

In [44]:
# ¿En qué ciudades debemos reforzar este posicionamiento?
# Análisis combinado 'Ciudad' 'Conocía_marca'
df[df['Conocía_marca']=='No']['Ciudad'].value_counts(normalize = True)

Turín        0.300829
Venecia      0.198630
Roma         0.109950
Florencia    0.101839
Génova       0.095350
Palermo      0.064167
Verona       0.044881
Pisa         0.038933
Milán        0.029200
Siena        0.016222
Name: Ciudad, dtype: float64

Sin embargo, en este caso cada porcentaje es calculado con respecto al total de encuestados que No conocían la marca. Por ejemplo, el 30% de Turín es calculado con respecto a las 5.553 personas que No conocían la marca.

Pero resulta más adecuado calcular los porcentajes con respecto al total de encuestados **en cada ciudad**:

In [45]:
# Conteo de personas que NO conocen la marca, por ciudad
no_conocian = df[df['Conocía_marca']=='No']['Ciudad'].value_counts()
no_conocian.sort_index()

Florencia     565
Génova        529
Milán         162
Palermo       356
Pisa          216
Roma          610
Siena          90
Turín        1669
Venecia      1102
Verona        249
Name: Ciudad, dtype: int64

In [46]:
# Conteo total de encuestados por ciudad
encuestados = df['Ciudad'].value_counts()
encuestados.sort_index()

Florencia     937
Génova        906
Milán         429
Palermo       566
Pisa          360
Roma         1508
Siena         175
Turín        2824
Venecia      1828
Verona        455
Name: Ciudad, dtype: int64

In [47]:
# Porcentajes
porcentajes = no_conocian.sort_index() / encuestados.sort_index()
porcentajes.sort_values(ascending = False)

Palermo      0.628975
Florencia    0.602988
Venecia      0.602845
Pisa         0.600000
Turín        0.591006
Génova       0.583885
Verona       0.547253
Siena        0.514286
Roma         0.404509
Milán        0.377622
Name: Ciudad, dtype: float64

> **Respuesta a la pregunta ¿en qué ciudades debemos reforzar este posicionamiento?**: al analizar en cada ciudad el porcentaje de encuestados que no conoce nuestra marca, se obtuvieron estos resultados:

> - En 8 de las 10 ciudades donde tenemos presencia (Palermo, Florencia, Venecia, Pisa, Turín, Génova, Verona, Siena) pues en dichas ciudades al menos el 50% de los encuestados no conocía nuestra marca

In [48]:
# ¿Por qué los consumidores prefieren otras marcas y no las nuestras?: 
# columna "Motivos_escoger_marca"
df['Motivos_escoger_marca'].value_counts(normalize=True)

Reputación de la marca    0.265318
El sabor                  0.200841
Fácil de encontrar        0.191129
Efectividad               0.174910
Otro                      0.167801
Name: Motivos_escoger_marca, dtype: float64

> **Respuesta a la pregunta ¿Por qué los consumidores prefieren otras marcas y no las nuestras?**:

> Principalmente porque: (1) Confían en una marca con mejor reputación (27%), (2) les gusta más el sabor de la bebida ofrecida por otras márcas (20%), (3) a que es fácil de encontrar (19%)

### 4.3. Canales de mercadeo y conocimiento de la marca

Las preguntas que queremos responder son:

 - ¿Cuál es el canal de mercadeo más efectivo?
 
Podríamos simplemente analizar la columna 'Canales_marketing':

In [49]:
df['Canales_marketing'].value_counts(normalize = True)

Publicidad online       0.402082
Publicidad en TV        0.268823
Vallas publicitarias    0.122647
Otro                    0.122347
Medios impresos         0.084101
Name: Canales_marketing, dtype: float64

Pero podemos ir un paso más allá y analizar, por ejemplo:

- Cuáles son los canales de mercadeo más efectivos **entre los que han probado nuestra marca**
- Cuáles son los canales de mercadeo más efectivos **por rangos de edad**

In [50]:
# Canales más efectivos entre quienes han probado nuestra marca
df[df['Conocía_marca']=='Sí']['Canales_marketing'].value_counts(normalize = True)

Publicidad online       0.406081
Publicidad en TV        0.282658
Vallas publicitarias    0.118018
Otro                    0.109910
Medios impresos         0.083333
Name: Canales_marketing, dtype: float64

In [51]:
# Rangos de edad
niv_edad = df['Edad'].value_counts().index
niv_edad

Index(['19-30', '31-45', '15-18', '46-65', '65+'], dtype='object')

In [52]:
# Canales más efectivos por rangos de edad
for niv in niv_edad:
    print(f'Rango de edad: {niv}')
    print(df[df['Edad']==niv]['Canales_marketing'].value_counts(normalize=True))
    print('-'*20)

Rango de edad: 19-30
Publicidad online       0.483040
Publicidad en TV        0.233630
Otro                    0.110103
Vallas publicitarias    0.105931
Medios impresos         0.067295
Name: Canales_marketing, dtype: float64
--------------------
Rango de edad: 31-45
Publicidad en TV        0.310708
Publicidad online       0.206155
Vallas publicitarias    0.181703
Otro                    0.171164
Medios impresos         0.130270
Name: Canales_marketing, dtype: float64
--------------------
Rango de edad: 15-18
Publicidad online       0.475454
Publicidad en TV        0.332213
Vallas publicitarias    0.078682
Otro                    0.063215
Medios impresos         0.050437
Name: Canales_marketing, dtype: float64
--------------------
Rango de edad: 46-65
Publicidad en TV        0.274648
Publicidad online       0.255869
Otro                    0.183099
Vallas publicitarias    0.152582
Medios impresos         0.133803
Name: Canales_marketing, dtype: float64
--------------------
Rango de eda

> **Respuesta a la pregunta ¿¿Cuál es el canal de mercadeo más efectivo?**

> Los canales más efectivos son la publicidad online y la TV. La publicidad online es el medio más efectivo para el rango de edades 15-30 años, mientras que para el rango 31-65+ o más es más adecuada la TV.

<a id='sec_5'></a>
## 5. Reporte final y recomendaciones

El reporte final podría ser un informe escrito o una breve presentación con elementos gráficos. En ambos casos el objetivo es sintetizar las observaciones y las respuestas a las preguntas, es decir presentar los *insights* resultado de este análisis.

Estos *insights* son de carácter **descriptivo**, mientras que en las recomendaciones debemos **extraer conclusiones** a partir de los *insights* para así generar un listado de sugerencias.

### 5.1. Reporte final

En este caso haremos un breve informe en formato de texto. Un ejemplo sería:

Tras el análisis realizado observamos que:

- El género que más consume nuestra bebida son los hombres y el grupo de edad con más consumidores de nuestro producto es de los 19 a los 30 años.
- Los líderes actuales del mercado son Coca Cola, Pepsi y Monster con un 65% de participación en el mercado. Nuestro producto se encuentra en el 5° lugar con un porcentaje de participación del 10%
- Con respecto al posicionamiento de la marca encontramos que:
    - Nuestra marca y logo no generan recordación en los compradores y que los consumidores no están del todo satisfechos con el sabor de la bebida.
    - En 8 de las 10 ciudades Italianas donde tenemos presencia, la gente NO conoce nuestra marca
    - La gente prefiere otras marcas por su reputación, por su disponibilidad y por su sabor.
- Los canales más efectivos son la publicidad online y la TV. La publicidad online es el medio más efectivo para el rango de edades 15-30 años, mientras que para el rango 31-65 o más es más adecuada la TV.

### 5.2. Recomendaciones

Con base en lo anterior se sugieren estas estrategias:

1. Mantener estrategias de publicidad dirigidas a hombres entre los 19 y 30 años, especialmente en la modalidad online y publicidad en medios de televisión dirigidas a los rangos de 31 a 65 o más años.
2. Reforzar las estrategias de publicidad dirigidas a  dos segmentos: (1) los géneros mujeres y no binario y (2) en las ciudades de Palermo, Florencia, Venecia, Pisa, Turín, Génova, Verona, Siena
3. Considerar la posibilidad de modificar el sabor de nuestra bebida para que esté alineado con las preferencias de los consumidores.
4. Fortalecer la cadena de distribución para garantizar que nuestra bebida esté disponible en las ciudades de interés.

<a id='sec_6'></a>
## 6. Sugerencias finales

En este proyecto hemos analizado sólo una parte del set de datos: la que guardaba relación directa con las preguntas planteadas al comienzo.

Sin embargo, sugiero complementar estos resultados analizando por ejemplo: 

- Preferencias del consumidor: ¿Cuáles son los ingredientes preferidos?, ¿Qué tipo de paquetes son los preferidos?
- Comportamientos de compra: ¿Dónde prefieren las personas comprar bebidas energizantes?, ¿Cuáles son las situaciones típicas de consumo de bebibas energéticas? ¿Qué factores influyen más en las decisiones de compra de las personas?
- Desarrollo de producto: ¿Qué área deberíamos priorizar para el desarrollo y mejora del producto?

Además, en lugar de un reporte en formato de texto se podría realizar una breve presentación incluyendo elementos gráficos (con ayuda de Matplotlib).